# Fisher Score- Chisquare Test For Feature Selection

In [1]:
import pandas as pd
import seaborn as sns
df = sns.load_dataset('titanic')

In [2]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


**Here There are so many categorical variables   
And the Chi square test is made for the categorical variables**
  
**Out of so many categorical variables lets select some of them in order to apply this method  
for x = ['sex','pclass','embarked','class','alone','deck','alive']  
for y = ['survived']  as it is the target variable**

In [4]:
X = df[['sex','pclass','embarked','class','alone','deck','alive']]
y = df['survived']

In [5]:
X.head()

,sex,pclass,embarked,class,alone,deck,alive
0,male,3,S,Third,False,NaN,no
1,female,1,C,First,False,C,yes
2,female,3,S,Third,True,NaN,yes
3,female,1,S,First,False,C,yes
4,male,3,S,Third,True,NaN,no


In [6]:
# lets do the label encoding now
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()


In [7]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   sex       891 non-null    object  
 1   pclass    891 non-null    int64   
 2   embarked  889 non-null    object  
 3   class     891 non-null    category
 4   alone     891 non-null    bool    
 5   deck      203 non-null    category
 6   alive     891 non-null    object  
dtypes: bool(1), category(2), int64(1), object(3)
memory usage: 31.1+ KB


In [8]:
for columns in X.columns:
    if X[columns].dtype == 'object' or X[columns].dtype == 'category' or X[columns].dtype == 'bool':
        X[columns] = labelencoder.fit_transform(X[columns])
X.head()

C:\Users\User\AppData\Local\Temp\ipykernel_13504\4060016538.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[columns] = labelencoder.fit_transform(X[columns])


,sex,pclass,embarked,class,alone,deck,alive
0,1,3,2,2,0,7,0
1,0,1,0,0,0,2,1
2,0,3,2,2,1,7,1
3,0,1,2,0,0,2,1
4,1,3,2,2,1,7,0


In [9]:
# Lets split the data then
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)
X_train.head()

,sex,pclass,embarked,class,alone,deck,alive
857,1,1,2,0,1,4,1
52,0,1,0,0,0,3,1
386,1,3,2,2,0,7,0
124,1,1,2,0,0,3,0
578,0,3,0,2,0,7,0


In [10]:
#Lets perform chi2 test
# this test will actually return 2 values
# 1 will be the f score note: higher the f score more the feature is important
# another will be p value and less the p value more the feature is important

In [11]:
from sklearn.feature_selection import chi2
f_p_values = chi2(X_train,y_train)
f_p_values

(array([ 63.55447864,  21.61080949,   6.2675888 ,  38.39825876,
          9.03328564,  37.02895489, 381.        ]),
 array([1.55992554e-15, 3.33964360e-06, 1.22966368e-02, 5.76838412e-10,
        2.65107556e-03, 1.16388007e-09, 7.53782890e-85]))

In [18]:
p_values = pd.Series(f_p_values[1])
p_values.index = X_train.columns
p_values


sex         1.559926e-15
pclass      3.339644e-06
embarked    1.229664e-02
class       5.768384e-10
alone       2.651076e-03
deck        1.163880e-09
alive       7.537829e-85
dtype: float64

In [22]:
p_values.sort_values(ascending=True)

alive       7.537829e-85
sex         1.559926e-15
class       5.768384e-10
deck        1.163880e-09
pclass      3.339644e-06
alone       2.651076e-03
embarked    1.229664e-02
dtype: float64

## Conclusion:
Here we can see the p value in the increasing order and we know that less the p value more important the features
Depending on that note we can select whatever the number of features we need in the problem